# Part 1: 2 Asset Optimization

1. Get data (yfinance/api)
2. find mean, stdev and correlation
3. create table for weightage
4. create MVP portfolio (solver) 
5. plot MVP line with CAL line
6. 

In [42]:
# Import Library
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sciop
from datetime import datetime
import os

In [43]:
# Create a matrix of stock return and standard deviation
asset_risk_free = 0.05
# Generating Trial Asset list
assetlist = ["US", "UK", "France" ,"Germany" ,"Japan" ]

# Generating Trial Asset Return Deviation Matrix

asset_risk_return_dict={"Return":{"US":0.1355, "UK":0.1589,"France":0.1519,"Germany":0.1435,"Japan":0.1497},
                        "Deviation":{"US":0.1535, "UK":0.2430,"France":0.2324,"Germany":0.2038,"Japan":0.2298},
                        }

# Generating Trial Asset Correlation Matrix
asset_correlation_matrix_dict={"US":{"US":1,"UK":0.5003,"France":0.4398,"Germany":0.3681,"Japan":0.2663},
                          "UK":{"US":0.5003,"UK":1,"France":0.5420,"Germany":0.4265,"Japan":0.3581},
                          "France":{"US":0.4398,"UK":0.5420,"France":1,"Germany":0.6032,"Japan":0.3923},
                          "Germany":{"US":0.3681,"UK":0.4265,"France":0.6032,"Germany":1,"Japan":0.3663},
                          "Japan":{"US":0.2663,"UK":0.3581,"France":0.3923,"Germany":0.3663,"Japan":1}}

# Saving Matrix into pd DataFrame
asset_list_df = pd.DataFrame(assetlist)
asset_risk_return= pd.DataFrame(asset_risk_return_dict)
asset_correlation_matrix= pd.DataFrame(asset_correlation_matrix_dict)


# Part 2: N Asset Optimization

In [44]:
# Finding Weightage of Portfolio on Global Minimum Variance

# Step 1: Extract standard deviations
std_devs = np.array([asset_risk_return_dict["Deviation"][asset] for asset in assetlist])

# Step 2: Create covariance matrix
correlation_matrix = asset_correlation_matrix.loc[assetlist, assetlist].values  # ensure correct order
cov_matrix = np.outer(std_devs, std_devs) * correlation_matrix

# Step 3: Define Minimum Variance Function
def global_min_var_portfolio(weights):
    return weights.T @ cov_matrix @ weights # Matrix multiplication to find variance

# Step 4: Define Constraints and Bounds
number_of_assets = len(assetlist) # asset count
init_guess = np.ones(number_of_assets)/number_of_assets # start with equal weightages
bounds = [(0,1) for a in range(number_of_assets)] # Bound the Asset weightage
constraints = {'type': 'eq', # Equality Constraint
               'fun' : lambda w : np.sum(w)-1 # Must Equal to 0
               # This constraint ensures that sum of all weight = 1
               # or sum of weight - 1 = 0
               }

# Step 5: Solve for Min Variance Portfolio
result = sciop.minimize(global_min_var_portfolio, init_guess, method='SLSQP',
                bounds=bounds, constraints=constraints)

# Step 6: Output Weightages
min_variance_weights = result.x

In [45]:
# Finding Weightage of Portfolio on Global Minimum Variance

# Step 1: Extract standard deviations
std_devs = np.array([asset_risk_return_dict["Deviation"][asset] for asset in assetlist])

# Step 2: Create covariance matrix
correlation_matrix = asset_correlation_matrix.loc[assetlist, assetlist].values  # ensure correct order
cov_matrix = np.outer(std_devs, std_devs) * correlation_matrix

# Step 3: Define Minimum Variance Function
def global_min_var_portfolio(weights):
    return weights.T @ cov_matrix @ weights # Matrix multiplication to find variance

# Step 4: Define Constraints and Bounds
number_of_assets = len(assetlist) # asset count
init_guess = np.ones(number_of_assets)/number_of_assets # start with equal weightages
bounds = [(0,1) for a in range(number_of_assets)] # Bound the Asset weightage
constraints = {'type': 'eq', # Equality Constraint
               'fun' : lambda w : np.sum(w)-1 # Must Equal to 0
               # This constraint ensures that sum of all weight = 1
               # or sum of weight - 1 = 0
               }

# Step 5: Solve for Min Variance Portfolio
result = sciop.minimize(global_min_var_portfolio, init_guess, method='SLSQP',
                bounds=bounds, constraints=constraints)

# Step 6: Output Weightages
min_variance_weights = result.x
MVP_portfolio = dict(zip(assetlist, min_variance_weights))
print(f'''
      MVE Portfolio Generated:
      {MVP_portfolio}
      ''')


      MVE Portfolio Generated:
      {'US': np.float64(0.6177970453959858), 'UK': np.float64(5.5366744353439095e-18), 'France': np.float64(2.500908352384497e-18), 'Germany': np.float64(0.20939435918063665), 'Japan': np.float64(0.17280859542337745)}
      


In [46]:
# Finding Weightage for Mean Variance Efficient Portfolio

# Step 1: Extract Mean 
assets_mean = np.array([asset_risk_return_dict["Return"][asset] for asset in assetlist])

# Step 2: Define Sharpe Formula
def negative_global_sharpe_portfolio(weights):
    mean_p = weights.T @ assets_mean
    variance = weights.T @ cov_matrix @ weights # Matrix multiplication to find variance
    return -(mean_p - asset_risk_free)/np.sqrt(variance)

# Step 3: Define Constraints and Bounds
number_of_assets = len(assetlist) # asset count
init_guess = np.ones(number_of_assets)/number_of_assets # start with equal weightages
bounds = [(0,1) for a in range(number_of_assets)] # Bound the Asset weightage
constraints = {'type': 'eq', # Equality Constraint
               'fun' : lambda w : np.sum(w)-1 # Must Equal to 0
               # This constraint ensures that sum of all weight = 1
               # or sum of weight - 1 = 0
               }

# Step 4: Solve for Min Variance Portfolio
result = sciop.minimize(negative_global_sharpe_portfolio, init_guess, method='SLSQP',
                bounds=bounds, constraints=constraints)

# Step 5: Output Weightages
max_sharpe_weights = result.x

MVE_portfolio = dict(zip(assetlist, max_sharpe_weights))
print(f'''
      MVE Portfolio Generated:
      {MVE_portfolio}
      ''')


      MVE Portfolio Generated:
      {'US': np.float64(0.5071417369229368), 'UK': np.float64(0.07470887972807165), 'France': np.float64(0.024715331355035768), 'Germany': np.float64(0.18943971928785783), 'Japan': np.float64(0.20399433270609799)}
      


### Obtaining Mean, Stdev, Sharpe and Covariance Matrix of MVE and MVP Portfolio 

In [52]:
# Find MVP's Mean, Stdev and Sharpe

# Mean
mvp_mean = min_variance_weights.T @ assets_mean
mve_mean = max_sharpe_weights.T @ assets_mean

# Variance
mvp_variance = min_variance_weights.T @ cov_matrix @ min_variance_weights 
mve_variance = max_sharpe_weights.T @ cov_matrix @ max_sharpe_weights

# Stdev
mvp_stdev = np.sqrt(mvp_variance)
mve_stdev = np.sqrt(mve_variance)

# Sharpe
mvp_sharpe = (mvp_mean-asset_risk_free )/ mvp_stdev
mve_sharpe = (mve_mean-asset_risk_free )/ mve_stdev

In [ ]:
# Find Covariance Matrix
mvp_mve_cov = min_variance_weights.T @ cov_matrix @ max_sharpe_weights
mvp_mve_cor = mvp_mve_cov / (mvp_stdev * mve_stdev)
